# 1. Extract only Encoder in detail (for fine-tuning)

>## 1) Setting & Data loading

In [1]:
!pip install laion-clap
!pip install torch
!pip install torchvision

In [2]:
import numpy as np
import librosa
import laion_clap
import torch.nn as nn
import os

In [3]:
dir(laion_clap)

['CLAP_Module',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'dir_path',
 'hook',
 'os',
 'sys']

In [4]:
dir(laion_clap.CLAP_Module)

['T_destination',
 '__annotations__',
 '__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_apply',
 '_call_impl',
 '_get_backward_hooks',
 '_get_backward_pre_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_maybe_warn_non_full_backward_hook',
 '_named_members',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_version',
 'add_module',
 'apply',
 'bfloat16',
 'buffers',
 'call_super_init',
 'children',
 'cpu',
 'cuda',
 'double',
 'dump_patches',
 'eval',
 'extra_repr',
 'float',
 'forward',
 'get_audio_embedding_from_data',
 'get

>## 2) Extract Audio Embedding
: 총 2000개 데이터 중 100개만 넣었음

In [6]:
import os

# 디렉토리 경로 지정
dir_path = "./data/ESC-50-master/audio"

# 디렉토리 내 파일 목록 불러오기
file_list = os.listdir(dir_path)

# 출력
print(file_list[:5])
print(len(file_list))

['1-13571-A-46.wav', '1-29532-A-16.wav', '5-260011-A-34.wav', '3-96606-B-49.wav', '5-160551-A-42.wav']
2000


In [7]:
# prefix 추가
prefix = './data/ESC-50-master/audio/'

audio_file = [prefix + item for item in file_list]

print(audio_file[:10]) 
print(len(audio_file))

['./data/ESC-50-master/audio/1-13571-A-46.wav', './data/ESC-50-master/audio/1-29532-A-16.wav', './data/ESC-50-master/audio/5-260011-A-34.wav', './data/ESC-50-master/audio/3-96606-B-49.wav', './data/ESC-50-master/audio/5-160551-A-42.wav', './data/ESC-50-master/audio/4-212698-A-39.wav', './data/ESC-50-master/audio/1-96890-A-37.wav', './data/ESC-50-master/audio/4-181286-A-10.wav', './data/ESC-50-master/audio/2-112213-A-39.wav', './data/ESC-50-master/audio/2-209475-A-25.wav']
2000


In [8]:
# quantization
def int16_to_float32(x):
    return (x / 32767.0).astype(np.float32)


def float32_to_int16(x):
    x = np.clip(x, a_min=-1., a_max=1.)
    return (x * 32767.).astype(np.int16)

model = laion_clap.CLAP_Module(enable_fusion=False)
model.load_ckpt() # download the default pretrained checkpoint.


/home/jeong_soonju/default/lib/python3.9/site-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]
Some weights of the model checkpoint at roberta-base were not used when initializing RobertaModel: ['lm_head.layer_norm.weight', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.bias', 'lm_head.bias', 'lm_head.decoder.weight']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForS

Load our best checkpoint in the paper.
The checkpoint is already downloaded
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm2.we

In [9]:
# Directly get audio embeddings from audio files

audio_embed = model.get_audio_embedding_from_filelist(x = audio_file[:100], use_tensor=False)
print(audio_embed[:,-20:])
print(audio_embed.shape)

[[ 0.03401422  0.06023627  0.07191774 ... -0.02979645 -0.04799221
  -0.0649235 ]
 [-0.01128098 -0.011688    0.08284086 ... -0.03771019  0.0148686
   0.01149836]
 [ 0.07900055  0.07890835  0.06254026 ... -0.07600399 -0.01828993
   0.00212966]
 ...
 [ 0.02528274  0.00458613  0.03585381 ...  0.00787576  0.03576995
   0.04022818]
 [ 0.01853131  0.03921445  0.02782208 ... -0.03592799  0.03818468
   0.04554203]
 [ 0.02675372  0.01565943  0.0220779  ... -0.00032908  0.03575645
   0.03166889]]
(100, 512)


In [10]:
# Get audio embeddings from audio data
audio_data, _ = librosa.load('./data/ESC-50-master/audio/1-13571-A-46.wav', sr=48000) # sample rate should be 48000
audio_data = audio_data.reshape(1, -1) # Make it (1,T) or (N,T)
audio_embed = model.get_audio_embedding_from_data(x = audio_data, use_tensor=False)
print(audio_embed[:,-20:])
print(audio_embed.shape)

[[ 0.03403534  0.06023425  0.07190047 -0.02228823 -0.03047081  0.01141005
  -0.0611472   0.00043566  0.00295208  0.07440427  0.09748671  0.08374038
   0.08714106  0.04601844 -0.04469379  0.03515762  0.03421513 -0.02977145
  -0.04800081 -0.06491148]]
(1, 512)


### 오디오 데이터가 너무 커서 임베딩할 때 커널이 죽음.. 그래서 100개만 슬라이스했음. 임베딩 할 때 어떻게 메모리 안터지고 불러올 수 있을까

>## 3) Extract Text Embedding
: 총 2000개 중 100개만 넣었음

In [11]:
import pandas as pd

# CSV 파일을 DataFrame으로 로드
df = pd.read_csv("./data/ESC-50-master/meta/esc50.csv")

# 'category' column의 모든 데이터를 list로 받음
category_data = df['category'].tolist()

# 출력
print(len(category_data))
print(category_data[:10])

2000
['dog', 'chirping_birds', 'vacuum_cleaner', 'vacuum_cleaner', 'thunderstorm', 'thunderstorm', 'door_wood_knock', 'can_opening', 'crow', 'door_wood_knock']


In [12]:
# Get text embedings from texts:
text_data = category_data
text_embed = model.get_text_embedding(text_data[:100])
print(text_embed)
print(text_embed.shape)

[[ 7.51032587e-03 -1.21430203e-03  1.29887294e-02 ...  2.76181791e-02
   7.74938148e-03  1.29231289e-02]
 [ 4.22641411e-02  8.13215151e-02 -1.10665656e-04 ... -2.58268900e-02
  -2.80980468e-02 -5.83497286e-02]
 [ 3.94157786e-03  2.36528050e-02 -1.77783612e-02 ...  5.98671585e-02
   3.54673155e-02  7.57417828e-03]
 ...
 [ 9.34299678e-02 -1.45519748e-02 -1.21943876e-01 ...  2.21117083e-02
  -3.05675492e-02 -3.61665636e-02]
 [-5.67686297e-02  4.95181903e-02  1.54218050e-02 ...  1.63239948e-02
   1.55661255e-02  7.64964102e-03]
 [ 1.31103229e-02 -1.66436210e-02  1.92866344e-02 ... -2.22927611e-02
  -4.94182855e-03 -5.51377088e-02]]
(100, 512)


In [14]:
# Compute cosine similarity
norm_text = np.linalg.norm(text_embed, axis=1, keepdims=True)
norm_audio = np.linalg.norm(audio_embed, axis=1, keepdims=True)
dot_product = np.dot(text_embed, audio_embed.T)
cos_similarity = dot_product / (norm_text * norm_audio.T)

print(cos_similarity)

[[ 0.01216932]
 [-0.03393425]
 [-0.1361347 ]
 [-0.1361347 ]
 [-0.0761677 ]
 [-0.0761677 ]
 [-0.02343627]
 [-0.17361444]
 [ 0.07817606]
 [-0.02343627]
 [-0.02343627]
 [ 0.09289318]
 [ 0.09289318]
 [ 0.09289318]
 [ 0.01216932]
 [ 0.09289318]
 [-0.0761677 ]
 [-0.05459817]
 [-0.05459819]
 [-0.05459819]
 [-0.05459819]
 [ 0.09289318]
 [ 0.09289318]
 [ 0.09289318]
 [ 0.0731846 ]
 [-0.00746779]
 [-0.11009359]
 [-0.08927881]
 [-0.03072681]
 [ 0.02010207]
 [-0.04015809]
 [-0.04015809]
 [-0.04015809]
 [ 0.45374185]
 [ 0.45374185]
 [ 0.12035933]
 [-0.09581985]
 [-0.07152376]
 [ 0.12035935]
 [ 0.03201603]
 [ 0.03201603]
 [ 0.03201603]
 [ 0.03201603]
 [ 0.03201603]
 [ 0.03201603]
 [-0.07007433]
 [-0.07007433]
 [-0.05459819]
 [-0.05459819]
 [ 0.13917448]
 [-0.04015809]
 [-0.11662252]
 [-0.11662252]
 [ 0.09437966]
 [-0.11471956]
 [-0.12006029]
 [-0.12006029]
 [-0.12006029]
 [-0.12006029]
 [-0.12006029]
 [-0.12006029]
 [-0.0531735 ]
 [ 0.03840921]
 [-0.11471956]
 [-0.1181232 ]
 [-0.11471956]
 [-0.11471

In [16]:
# ESC50_Model_Cosine_Similarlity [:100]

# Compute row-wise mean
row_means = np.mean(cos_similarity, axis=1)
# Compute overall mean
overall_mean = np.mean(row_means)

print("Overall mean cosine similarity:", overall_mean)

Overall mean cosine similarity: -0.016434615


# 2. Pretrained Models

>## 1) Load Model (Dataset : ESC50)

In [ ]:
import laion_clap
import glob
import json
import torch
import numpy as np

# device = torch.device('cuda:0')
# download https://drive.google.com/drive/folders/1scyH43eQAcrBz-5fAw44C6RNBhC3ejvX?usp=sharing and extract ./ESC50_1/test/0.tar to ./ESC50_1/test/

# Pick a Dataset type
esc50_test_dir = 'data/0.tar/*/' # (flac-json)
class_index_dict_path = 'data/ESC50_class_labels_indices_space.json'

# Load the model
model = laion_clap.CLAP_Module(enable_fusion=True) # device=device
model.load_ckpt()

In [ ]:
# Get the class index dict
class_index_dict = {v: k for v, k in json.load(open(class_index_dict_path)).items()}
print('class 갯수:',len(class_index_dict))
for key, value in list(class_index_dict.items())[:10]:
    print(key, value)

>## 2) Load Audio / Text / Ground-truth files

In [ ]:
# Get all the data
import tarfile

# 압축 파일 해제
with tarfile.open('data/0.tar') as tar:
    tar.extractall('data/ESC50_1/test/')

# flac 파일 경로 가져오기
audio_files = sorted(glob.glob('data/ESC50_1/test/**/*.flac', recursive=True))

print('audio files 갯수:',len(audio_files)) # 여러개 파일 
print(audio_files[:3])

In [ ]:
json_files = sorted(glob.glob('./data/ESC50_1/test/**/*.json', recursive=True))
print('json_files 갯수:',len(json_files))
print(json_files[:3])

In [ ]:
ground_truth_idx = [class_index_dict[json.load(open(jf))['category']] for jf in json_files]
print('ground_truth_idx 갯수:',len(ground_truth_idx))
print(ground_truth_idx[:20])

>## 3) Zeroshot Classification

In [ ]:
import torchaudio

torchaudio.__version__

>>### (1) Audio/Text Embedding 값 불러오기 & 간단스크립트

In [ ]:
with torch.no_grad():
    ground_truth = torch.tensor(ground_truth_idx).view(-1, 1)

    # Get text features
    all_texts = ["This is a sound of " + t for t in class_index_dict.keys()]
    text_embed = model.get_text_embedding(all_texts)
    audio_embed = model.get_audio_embedding_from_filelist(x=audio_files)
   

- ground_truth_idx 리스트(classification 답)를 PyTorch 텐서로 변환하고,.view로 차원을 맞춰둠.
- all texts 인자에 class_index_dict의 key값(text)를 받아 확인할 수 있게 하고,
- text인 all texts 리스트에 대한 임베딩 값 text_embed 얻고,
- audio files 리스트에 있는 모든 오디오 파일에 대한 audio_embed 얻음.


In [ ]:
    print('text_embed length :',len(text_embed)) # class 갯수
    print('text_embed shape :',text_embed.shape)
    print(text_embed[:1])

In [ ]:
    print('audio_embed length :',len(audio_embed)) # 오디오파일 갯수
    print('audio_embed shape :',audio_embed.shape)
    print(audio_embed[:1])

>>### (2) Contrastive Learning & Prediction

In [ ]:
    ranking = torch.argsort(torch.tensor(audio_embed) 
                            @ torch.tensor(text_embed).t(), 
                            descending=True)
    

1. audio_embed & text_embed.T 내적곱해서 latent space 통일
2. torch.argsort : 계산한 내적값 작은순으로 sort해서 다시 index 부여
3. 다시 내림차순으로 정렬  
**>>> ranking = 음성과 텍스트 데이터간 유사도가 높은 순서대로 정렬된 인덱스가 담김**    

In [ ]:
    preds = torch.where(ranking == ground_truth)[1]
    preds = preds.cpu().numpy()

1. 가장 유사하다고 판단한 ranking 값과 실제답 ground_truth가 같으면 true를 반환함
2. true값의 위치를 찾아 주는 것이 torch.where
3.  [1] = 인덱스값   
**>>> preds = 음성과 텍스트간의 유사도가 가장 높은 class name(text)를 담음**
4. tensor를 np로 변환

>>### (3) Results

In [ ]:
    metrics = {}
    metrics[f"mean_rank"] = preds.mean() + 1
    metrics[f"median_rank"] = np.floor(np.median(preds)) + 1

In [ ]:
    print(metrics[f"mean_rank"])
    print(metrics[f"median_rank"])

1. 결과값을 담을 metrics dictionary 열어두고
2. preds의 평균값과 중앙값을 구해서 순위를 0이 아닌 1부터 시작하게함

In [ ]:
    for k in [1, 5, 10]:
        metrics[f"R@{k}"] = np.mean(preds < k)
    # map@10
    metrics[f"mAP@10"] = np.mean(np.where(preds < 10, 1 / (preds + 1), 0.0))

- R@1,5,10 = 예측된 순위(preds)가 실상위 k개 안에 있으면 1, 아니면 0 한 뒤 평균
- mAP@10 = Mean Average Precision at 10 : 모델이 상위 10개 예측에 대해 얼마나 정확한 결과를 내는지 평가하는 지표
    

In [ ]:
    print(
        f"Zeroshot Classification Results: "
        + "\t".join([f"{k}: {round(v, 4):.4f}" for k, v in metrics.items()])
    )

- mean_rank: 1.2425 : 모델이 예측한 결과 중 상위 2개 클래스가 정답 클래스로 예측되었음.
- median_rank: 1.0 : 중앙값으로 예측해도 평균적으로 1이라는 것은 대체로 예측된 클래스 중 1위 클래스가 실제값이라는 뜻.
- R@1,5,10 : 예측한 1,5,10위까지의 순위 안에 정답이 포함될 확률

        

>>### (4) Visualization

In [ ]:
print(type(ranking))
print(type(ground_truth))

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

def confusion_matrix(y_true, y_pred, num_classes):
    # num_classes x num_classes 크기의 0으로 채워진 행렬 생성
    conf_mat = np.zeros((num_classes, num_classes))

    # 행렬의 각 [i][j] 요소는 i번 클래스가 j번 클래스로 분류된 샘플의 수
    for i in range(len(y_true)):
        true_idx = y_true[i]
        pred_idx = y_pred[i]
        conf_mat[true_idx][pred_idx] += 1

    return conf_mat

def plot_confusion_matrix(conf_mat, class_names):
    disp = ConfusionMatrixDisplay(confusion_matrix=conf_mat, display_labels=class_names)
    fig, ax = plt.subplots(figsize=(8, 8))
    disp = disp.plot(include_values=True, cmap='Blues', ax=ax, xticks_rotation=90)  # x축 레이블을 90도 기울임
    plt.show()

# confusion matrix 생성
num_classes = len(class_index_dict)
conf_mat = confusion_matrix(ground_truth, torch.argmax(ranking, dim=1), num_classes)

# 클래스 이름 가져오기
class_names = list(class_index_dict.keys())

# 시각화
plot_confusion_matrix(conf_mat, class_names)


hand saw - dog(8) / cat - hand saw(6) / chainsaw - rooster(6) 

In [ ]:
!pip install pydub

In [ ]:
from pydub import AudioSegment
from pydub.playback import play

wav_path = './data/dog.wav'
audio = AudioSegment.from_file(wav_path, format="wav")
play(audio)

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import ConfusionMatrixDisplay

def confusion_matrix(y_true, y_pred, num_classes):
    # num_classes x num_classes 크기의 0으로 채워진 행렬 생성
    conf_mat = np.zeros((num_classes, num_classes))

    # 행렬의 각 [i][j] 요소는 i번 클래스가 j번 클래스로 분류된 샘플의 수
    for i in range(len(y_true)):
        true_idx = y_true[i]
        pred_idx = y_pred[i]
        conf_mat[true_idx][pred_idx] += 1

    return conf_mat

def plot_confusion_matrix(conf_mat, class_names):
    fig, ax = plt.subplots()
    im = ax.imshow(conf_mat, interpolation='nearest', cmap='Blues')
    
    # x, y 축 설정
    ax.set(xticks=np.arange(conf_mat.shape[1]), yticks=np.arange(conf_mat.shape[0]),
           xticklabels=class_names, yticklabels=class_names)
    
    # x 축 레이블 90도 회전
    plt.setp(ax.get_xticklabels(), rotation=90, ha="right", rotation_mode="anchor")
    
    # 각 셀에 값 출력
    for i in range(conf_mat.shape[0]):
        for j in range(conf_mat.shape[1]):
            text = ax.text(j, i, conf_mat[i, j], ha="center", va="center", color="w")
    
    # colorbar 추가
    cbar = ax.figure.colorbar(im, ax=ax)
    
    # 그래프 보여주기
    fig.tight_layout()
    plt.show()

# confusion matrix 생성
num_classes = len(class_index_dict)
conf_mat = confusion_matrix(ground_truth, torch.argmax(ranking, dim=1), num_classes)

# 클래스 이름 가져오기
class_names = list(class_index_dict.keys())

# 시각화
plot_confusion_matrix(conf_mat, class_names)


In [ ]:
stop

In [ ]:
num_classes = len(class_index_dict)
conf_mat = confusion_matrix(ground_truth, ranking, num_classes)


In [ ]:
from sklearn.metrics import multilabel_confusion_matrix

# 오차행렬 생성
conf_mat = multilabel_confusion_matrix(ground_truth, ranking)

# 클래스 이름 가져오기
class_names = list(class_index_dict.keys())

# 시각화
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap='Blues', xticklabels=class_names, yticklabels=class_names)
plt.ylabel('Actual')
plt.xlabel('Predicted')
plt.show()


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# 예측값과 실제값이 일치하는지 계산
correct_preds = (ranking == ground_truth)
print(correct_preds.shape)
incorrect_preds = (ranking != ground_truth)
incorrect_ranking = ranking[incorrect_preds][0]
print(incorrect_preds.shape)

In [ ]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix

# 오차행렬 생성
conf_mat = confusion_matrix(ground_truth[incorrect_preds], ranking[incorrect_preds])

# 클래스 이름 가져오기
class_names = list(class_index_dict.keys())

# 오차행렬 시각화
fig, ax = plt.subplots(figsize=(10, 10))
sns.heatmap(conf_mat, annot=True, fmt='d', cmap=plt.cm.Blues, cbar=False, ax=ax)

# x축, y축 라벨 설정
ax.set_xlabel('Predicted labels', fontsize=16)
ax.set_ylabel('True labels', fontsize=16)
ax.set_xticklabels(class_names, fontsize=12)
ax.set_yticklabels(class_names, fontsize=12)

# 그래프 제목 설정
ax.set_title('Confusion Matrix', fontsize=20)
plt.show()


In [ ]:
stop

In [ ]:
# 일치하는 경우 0, 그렇지 않은 경우 1 array 만들기
color_indices = np.where(correct_preds, 0, 1)
color_indices

In [ ]:
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(30, 100))
im = ax.imshow(color_indices, cmap="viridis")

ax.set_xticks(np.arange(color_indices.shape[1]))
ax.set_yticks(np.arange(color_indices.shape[0]))
ax.set_xticklabels(class_index_dict, fontsize=8)
ax.set_yticklabels(range(400), fontsize=8)

plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
plt.title("Confusion matrix")
plt.show()


In [ ]:
class_index_dict

In [ ]:
ranking

In [ ]:
a= [class_index_dict[i] for i in preds]
a

In [ ]:
import pandas as pd
import json

class_index_dict_path = 'data/ESC50_class_labels_indices_space.json'
class_index_dict = {v: k for v, k in json.load(open(class_index_dict_path)).items()}

df = pd.DataFrame({
    'preds': preds,
    'ground_truth': ground_truth.squeeze().tolist(),
    'audio_file': audio_files,
    'actual_class': [class_index_dict[i] for i in ground_truth.squeeze().tolist()],
    'predicted_class': [class_index_dict[i] for i in preds]
})

df_diff = df[df['actual_class'] != df['predicted_class']]
display(df_diff.head())
